In [1]:
import clean_data
import feature_selection as fs
import modeling_2 as model

# Clean Data

In [2]:
df = clean_data.runAll()

/Users/williamegan/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/Users/williamegan/anaconda3/lib/python3.7/site-packages/sklearn/utils/extmath.py:747: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/williamegan/anaconda3/lib/python3.7/site-packages/sklearn/utils/extmath.py:688: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


# Feature Selection

In [6]:
target = 'MD_EARN_WNE_P6'
df_no_id = df.drop('UNITID', axis=1)
n_features_to_select = 20
sel1 = fs.F_REGRESSION
sel2 = fs.MUTUAL_REGRESSION

In [ ]:
#F-SELECTION
f_sel_df = fs.selectFeatures(df_no_id, target, n_features_to_select, sel1)

In [ ]:
#MUTUAL SELECTION
mut_sel_df = fs.selectFeatures(df_no_id, target, n_features_to_select, sel2)

# Sampling and Splitting the Data for Each of the Feature Sets

In [10]:
#Sampling and Splitting for f_regression
train, test = model.sampling_data(f_sel_df)
x_train, y_train, x_test, y_test = model.split_data(train, test, target)

In [ ]:
#Sampling and Splitting for mutual_selection
train, test = model.sampling_data(mut_sel_df)
x_train, y_train, x_test, y_test = model.split_data(train, test, target)

# Run Models 

## Stage 1: f_regression features

### Random Forest

In [ ]:
est = 20
sel = model.RF

In [ ]:
rf_feature_importance, rf, rf_preds = model.run_model(x_train, y_train, x_test, est, sel)

### SVR

In [ ]:
est = .01
sel = model.SV

In [ ]:
svm, svr_preds = model.run_model(x_train, y_train, x_test, est, sel)

### Linear Regression

In [ ]:
est = 0
sel = model.LINEAR_R

In [ ]:
regr, linear_preds = model.run_model(x_train, y_train, x_test, est, sel)

### Lasso Regression

In [ ]:
est = 0
sel = model.LASSO_R

In [ ]:
lasso, lasso_preds = model.run_model(x_train, y_train, x_test, est, sel)

We run each of these twice and save them each in their own variables

In [ ]:
#run this after f_reg
f_sel_RF = rf_preds
f_sel_SVR = svr_preds
f_sel_Lin = linear_preds
f_sel_Lass = lasso_preds

In [ ]:
#run this after mutual_sel
mut_sel_RF = rf_preds
mut_sel_SVR = svr_preds
mut_sel_Lin = linear_preds
mut_sel_Lass = lasso_preds

NOW, we compare each of these to the truth using MSE and find the smallest one.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
errs_dict = {}
errs_dict['f_sel_RF_err'] =  mean_squared_error(y_test, f_sel_RF ) 
errs_dict['f_sel_SVR_err'] = mean_squared_error(y_test, f_sel_SVR )
errs_dict['f_sel_Lin_err'] = mean_squared_error(y_test, f_sel_Lin )
errs_dict['f_sel_Lass_err'] = mean_squared_error(y_test, f_sel_Lass)

errs_dict['mut_sel_RF_err'] = mean_squared_error(y_test, mut_sel_RF )
errs_dict['mut_sel_SVR_err'] = mean_squared_error(y_test, mut_sel_SVR )
errs_dict['mut_sel_Lin_err'] = mean_squared_error(y_test, mut_sel_Lin )
errs_dict['mut_sel_Lass_err'] = mean_squared_error(y_test, mut_sel_Lass)

In [ ]:
errs = [f_sel_RF_err, f_sel_SVR_err, f_sel_Lin_err, f_sel_Lass_err,  
        mut_sel_RF_err, mut_sel_SVR_err, mut_sel_Lin_err, mut_sel_Lass_err]


for d in errs:
    for i in d


winner = min(errs)
winner_name = ''

for err in errs:
    if err == winner:
        winner_name

    

# Time for Tuning (we do this only for our favorite model)

### SVM


we feed it: 
- the yummy x_train, y_train
- the usual target
- k to be 5 or 10
- cs is a list exponents of 10
- error metric is either model.MAE or model.MSE
'

Once the hungry boi has been fed, it will give us a dictionary with `len(cs)` keys, each containing `k` values. We will avergae the `k` values 

In [11]:
#Sample Input
k=5
cs = [10**i for i in range(-2,2)]
error_metric = model.MSE

In [ ]:
model.xValSVR(x_train, y_train, target, k, cs, error_metric)

c =  0.01 predicted on the  1 th fold!!
c =  0.1 predicted on the  1 th fold!!


Once we get this dictionary, we take the average across the folds for each hyperparameter. The hyperparameter average with the lowest error will be chosen.

### Random Forest

We feed it:
- x_train, y_train,
- usual target
- est_list, a list of numbers of estimators
- error_metric, model.MSE or model.MAE

In [3]:
#Sample Input
k=5
est_list = [10, 25, 50, 100, 200]
error_metric = model.MSE

In [ ]:
xValRF(x_train, y_train, target, k, est_list, error_metric)

### Lasso Regression

### Linear Regression

# 